In [154]:
import pandas as pd
import numpy as np

# 표준화/정규화
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 데이터 분리
from sklearn.model_selection import train_test_split

# 모델 가져오기
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# 모델 성능 평가 (roc_auc_score는 이진분류일 때만 사용 가능!)
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

## 제 2유형 연습하기
### 와인 종류 분류

#### 데이터 분석 순서
1. 라이브러리 및 데이터 확인
2. 데이터 탐색(EDA)
3. 데이터 전처리 및 분리
4. 모델링 및 성능 평가
5. 예측값 제출

In [155]:
from sklearn.datasets import load_wine
wine = load_wine()
x = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.DataFrame(wine.target)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2023)

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
y_train = pd.DataFrame(y_train)

x_test.reset_index()
y_train.columns = ['target']


### 참고사항 ###
# y_test는 실기 문제상에 주어지지 않음!!!

# Tip : X를 대문자로 쓰지말고 소문자로 쓸 것! (시험에서 실수가 빈번하게 발생)
# (참고 : 보통 X는 2차원 배열(행렬)이기 때문에 대문자로 쓰고, y는 1차원 배열(벡터)이기 때문에 소문자로 쓴다)

# (참고) 실기시험 데이터 형식 (실제 시험장에서 다를 수 있으니 반드시 체크)
# X_test = pd.read_csv("data/X_test.csv")
# X_test = pd.read_csv("data/X_train.csv")
# y_train = pd.read_csv("data/y_train.csv")

#### 데이터의 결측치, 이상치에 대해 처리하고 분류 모델을 사용하여 정확도, F1 score, AUC 값을 산출

#### 제출은 result 변수에 담아 양식에 맞게 제출하시오.

In [156]:
# 데이터 설명

print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

#### 01. 데이터 탐색(EDA)

In [157]:
# 데이터 행, 열 확인

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(142, 13)
(36, 13)
(142, 1)


In [158]:
# 초기 데이터 확인

print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
52     13.82        1.75  2.42               14.0      111.0           3.88   
146    13.88        5.04  2.23               20.0       80.0           0.98   
44     13.05        1.77  2.10               17.0      107.0           3.00   

     flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
52         3.74                  0.32             1.87             7.05  1.01   
146        0.34                  0.40             0.68             4.90  0.58   
44         3.00                  0.28             2.03             5.04  0.88   

     od280/od315_of_diluted_wines  proline  
52                           3.26   1190.0  
146                          1.33    415.0  
44                           3.35    885.0  
     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
168    13.58        2.58  2.69               24.5      105.0           1.55   
144    12.25        

In [159]:
# 변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인

print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 52 to 115
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       142 non-null    float64
 1   malic_acid                    142 non-null    float64
 2   ash                           142 non-null    float64
 3   alcalinity_of_ash             142 non-null    float64
 4   magnesium                     142 non-null    float64
 5   total_phenols                 142 non-null    float64
 6   flavanoids                    142 non-null    float64
 7   nonflavanoid_phenols          142 non-null    float64
 8   proanthocyanins               142 non-null    float64
 9   color_intensity               142 non-null    float64
 10  hue                           142 non-null    float64
 11  od280/od315_of_diluted_wines  142 non-null    float64
 12  proline                       142 non-null    float64
dtypes: f

In [160]:
# x_train과 x_test 데이터의 기초통계량 비교

print(x_train.describe().T)
print(x_test.describe().T)
print(y_train.describe().T)

                              count        mean         std     min       25%  \
alcohol                       142.0   13.025915    0.812423   11.03   12.3700   
malic_acid                    142.0    2.354296    1.142722    0.74    1.6100   
ash                           142.0    2.340211    0.279910    1.36    2.1900   
alcalinity_of_ash             142.0   19.354225    3.476825   10.60   16.8000   
magnesium                     142.0   98.732394   13.581859   70.00   88.0000   
total_phenols                 142.0    2.303592    0.633955    0.98    1.7575   
flavanoids                    142.0    2.043592    1.033597    0.34    1.2275   
nonflavanoid_phenols          142.0    0.361479    0.124627    0.14    0.2700   
proanthocyanins               142.0    1.575070    0.576798    0.41    1.2425   
color_intensity               142.0    5.005070    2.150186    1.28    3.3000   
hue                           142.0    0.950394    0.220736    0.54    0.7825   
od280/od315_of_diluted_wines

In [161]:
# y 데이터 확인

print(y_train.head())
print(y_train.value_counts())

     target
52        0
146       2
44        0
67        1
43        0
target
1         57
0         47
2         38
dtype: int64


#### 02. 데이터 전처리 및 분리

##### 1) 결측치, 2) 이상치, 3) 변수 처리

In [162]:
# 결측치 확인

print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
dtype: int64
alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
dtype: int64
target    0
dtype: int64


In [163]:
# 결측치 제거

# 만약 결측치가 있는 경우 dropna() 사용
# Default : 행을 기준으로 삭제!
# df = df.dropna()
# print(df)
# print(df.dropna().shape)

In [164]:
# 결측치 대체(평균값, 중앙값, 최빈값)

# 연속형의 경우 이상값에 영향이 가장 적은 중앙값 활용
# 범주형의 경우 최빈값을 활용
# 평균(mean), 중앙값(median), 최빈값(mode)
# ex> df['예시'].median()

# df['예시'] = df['예시'].fillna(대체값)

In [165]:
# 변수 처리

# 불필요한 변수 제거
# 방법1. df = df.drop(columns= ['변수1', '변수2'])
# 방법2. df = df.drop(['변수1', '변수2'], axis=1)

# 필요시 변수 추가 (파생변수 생성)
# ex> df['파생변수'] = df['A']*df['B'] (파생변수 생성 형태)

# 원핫인코딩 (가변수 처리)
# x_train = pd.get_dummies(x_train)
# x_test = pd.get_dummies(x_test)
# y_train = pd.get_dummies(y_train)
# print문으로 확인

#### 03. 데이터 분리

In [166]:
# 데이터 훈련 세트와 검증용 세트로 분할 (80% 훈련, 20% 검증)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['target'], test_size=0.2, stratify=y_train['target'], random_state=2023)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(113, 13)
(29, 13)
(113,)
(29,)


#### 04. 모델링 및 성능평가

In [167]:
# 랜덤 포레스트 사용
# 회귀 : RandomForestRegressor, 분류 : RandomForestClassifier

model = RandomForestClassifier()

model.fit(x_train, y_train)

RandomForestClassifier()

In [168]:
# 모델을 사용하여 테스트 데이터 예측

y_pred = model.predict(x_val)

In [169]:
# 모델 성능 평가 (정확도 : accuracy_score, 민감도(재현율) : recall_score, 특이도 : precision_score, ROC-AUC : roc_auc_score (이진 분류 때만 사용 가능!))

acc = accuracy_score(y_val, y_pred)   # 순서 주의  (실제값, 예측값)
f1 = f1_score(y_val, y_pred, average='macro')   # 순서 주의 (실제값, 예측값)

print("정확도 : ", acc)
print("macro f1 score : ", f1)

정확도 :  1.0
macro f1 score :  1.0


#### 05. 예측값 제출

##### 주의! test셋을 모델에 넣어 나온 예측값을 제출해야함!!

In [170]:
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({"result": y_result}).to_csv('수험번호.csv', index=False)

# 모델을 사용하여 테스트 데이터 예측

# 1. 특정 클래스로 분류할 경우 (predict)
y_result = model.predict(x_test)
print(y_result[:5])

# 2. 특정 클래스로 분류될 확률을 구할 경우 (predict_proba)
y_result_proba = model.predict_proba(x_test)
print(y_result_proba[:5])

[2 2 2 0 1]
[[0.02 0.   0.98]
 [0.07 0.09 0.84]
 [0.   0.11 0.89]
 [0.98 0.01 0.01]
 [0.03 0.91 0.06]]


In [171]:
result_prob = pd.DataFrame({
  'result' : y_result,
  'prob_0' : y_result_proba[:,0],
  'prob_1' : y_result_proba[:,1],
  'prob_2' : y_result_proba[:,2]
})

# class가 0일 확률 : prob_0
# class가 1일 확률 : prob_1
# class가 2일 확률 : prob_2

result_prob.head()

,result,prob_0,prob_1,prob_2
0,2,0.02,0.00,0.98
1,2,0.07,0.09,0.84
2,2,0.00,0.11,0.89
3,0,0.98,0.01,0.01
4,1,0.03,0.91,0.06


In [173]:
# 데이터 저장
# pd.DataFrame({"result": y_result}).to_csv('수험번호1.csv', index=False)

# tip! 데이터를 저장한 다음 불러와서 제대로 제출했는지 확인해보자
df_check = pd.read_csv('수험번호.csv')
print(df_check.head())

   result
0       2
1       2
2       2
3       0
4       1
